## **2. Prétraitement**
- Segmentation (phrases)
- Tokenization (mots)
- Étiquetage morphosyntaxique (POS Tagging) 
- (Lemmatisation)
- Filtrage (stopwords)
- Extraction de termes complexes (MWE / n-grammes / segments répétés)
- Chunking / Filtrage par patrons syntaxiques (basés sur les patrons fréquents dans les MeSH)
- Extraction de collocations significatives (en fonction du Log-likelihood ratio)
- (Extraction de concordances (KWIC) pour un ensemble de mots-clés d'intérêt)
- (Extraction de termes MeSH et SNOMED présents dans les données)

### **Lire le corpus** 

In [76]:
lng = 'fr'
acteur = 'msss'
tag = ''
if tag:
    file = acteur + '/' + acteur + '_' + tag + '.csv'

else:
    if lng == 'fr':
        file = acteur +'.csv'
    if lng == 'en':
        file = acteur + '_en.csv'

In [77]:
import shutil, re, random
from os import listdir, chdir, path
from pathlib import Path
from pandas import *

import nltk
#nltk.download(['popular'])
from nltk.tokenize import RegexpTokenizer
tokenizer_re = RegexpTokenizer(r"\w\'|\w+")
from nltk import bigrams, trigrams, ngrams, everygrams
from nltk.probability import FreqDist


import treetaggerwrapper
tagger = treetaggerwrapper.TreeTagger(TAGLANG=lng)


from collections import Counter
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from scipy.stats import binom, chi2

In [78]:
def lire_corpus(csv_file, langue=lng):
    base_path = '../03-corpus/2-data/'
    file_path = path.join(base_path, '1-' + langue, csv_file)

    #encoding='ISO 8859-1'
    with open(file_path, "r", encoding='utf-8') as f:
        data = read_csv(file_path, encoding='utf-8', sep=',')
        data = data[~data["url"].str.contains('pdf')] #  Problèmes 
        data = data['text']
    return data

In [79]:
data = lire_corpus(file) 
nb_docs = len(data)
print("On a un corpus de {} documents.".format(nb_docs))

On a un corpus de 4859 documents.


### **Nettoyage**

In [80]:
punct = '[!#$%&\(\)•►*+,-\/:;<=>?@[\]^_{|}~©«»—“”–—]'
spaces = '\s+'
postals = '([a-zA-Z]+\d+|\d+[a-zA-Z]+)+'
# phones = '\d{3}\s\d{3}-\d{4}' #très simple (trop)

text = [str(t).strip('\n').lower().replace('’', '\'') for t in data]
text = [re.sub(spaces, ' ', t) for t in text]
text = [re.sub('\d', 'MWE_STOP', t) for t in text] # On remplace les chiffres par un "stopword" pour ne pas briser de structures syntaxiques
text = [re.sub(postals, ' ', t) for t in text]
text = [re.sub(punct, ' ', t) for t in text]
text = [t.replace("  ", " " ) for t in text]

In [81]:
nb_docs = len(text)
print("On a un corpus de {} documents.".format(nb_docs))

On a un corpus de 4859 documents.


### **Extraire un échantillon aléatoire**

Sinon, on n'arrive pas à traiter la totalité du corpus pour des raisons de performance

In [82]:
def sample_corpus(corpus, ratio):
    n = round(ratio * len(corpus))
    corpus = random.sample(text, n)
    print("On va travailler sur un échantillon correspondant à environ " + str(ratio * 100) + " % des documents du corpus, soit {} documents". format(len(corpus)))
    return " ".join(corpus)
    
corpus = sample_corpus(text, 1)

On va travailler sur un échantillon correspondant à environ 100 % des documents du corpus, soit 4859 documents


### **Filtrage (MWE - stopwords formés de plusieurs tokens)**
Surtout pour filtrer les expressions relatives à l'architecture d'information / navigation Web

In [83]:
def filter_mwesw(corpus):
    file_mwesw = '../04-filtrage/mwe_stopwords.txt'
    with open (file_mwesw, 'r', encoding='utf-8') as f:
        mwe_sw = [t.lower().strip('\n') for t in f.readlines()]
    for mwe in mwe_sw:
        corpus = corpus.replace(mwe, ' MWE_STOP ').replace('  ', " ")
    return corpus

In [84]:
corpus = filter_mwesw(corpus)

### **Tokenisation / POS tagging** (TreeTagger)  
https://github.com/miotto/treetagger-python/blob/master/README.rst  
https://treetaggerwrapper.readthedocs.io/en/latest/

In [85]:
# Ici, on tokenise une première fois avec le Regex Tokenizer de NLTK pour "forcer" le Tree Tagger à tokeniser comme on veut
def tok(corpus):
    # Seulement les caractères alphabétiques
    tokens = tokenizer_re.tokenize(corpus)
    print("Avec le RegExpTokenizer, notre corpus contient {} tokens.".format(len(tokens)))
    temps = round(len(tokens) / 15000 / 60)
    print('Le POS tagging devrait prendre environ {} minutes.'.format(temps))
    return tokens

tokens = tok(corpus)
corpus = " ".join(tokens).replace("' ", "'")

Avec le RegExpTokenizer, notre corpus contient 3668836 tokens.
Le POS tagging devrait prendre environ 4 minutes.


In [86]:
def tagging(corpus):
    output = []
    for t in tagger.tag_text(corpus):
        try: 
            output.append((t.split('\t')[0], t.split('\t')[1]))
        except Exception as e:
            output.append(('MWE_STOP', 'NAM'))

    return output


In [87]:
tagged = tagging(corpus)
tokens = [t[0] for t in tagged]
tagged[:25]

[('MWE_STOP', 'NAM'),
 ('MWE_STOP', 'NAM'),
 ('MWE_STOP', 'NAM'),
 ('MWE_STOP', 'NAM'),
 ('recherche', 'VER:pres'),
 ('MWE_STOP', 'NAM'),
 ('MWE_STOP', 'NAM'),
 ('répertoire', 'NOM'),
 ('des', 'PRP:det'),
 ('indicateurs', 'NOM'),
 ('de', 'PRP'),
 ('gestion', 'NOM'),
 ('en', 'PRP'),
 ('santé', 'NOM'),
 ('et', 'KON'),
 ('services', 'NOM'),
 ('sociaux', 'ADJ'),
 ('MWE_STOP', 'NOM'),
 ('répertoires', 'NOM'),
 ('répertoire', 'NOM'),
 ('des', 'PRP:det'),
 ('indicateurs', 'NOM'),
 ('de', 'PRP'),
 ('gestion', 'NOM'),
 ('le', 'DET:ART')]

In [88]:
# Si on veut lemmatiser
### **Mapping POS Tags** (FRMG)

#Pour utiliser adéquatement notre lemmatiseur par la suite (FrenchLefffLemmatizer), on va mapper les étiquettes morphosyntaxiques du TreeTagger à celles que prend le lemmatiseur (celles issues de FRMG)

#http://alpage.inria.fr/frmgwiki/content/tagset-frmg

#file_path = '../04-filtrage/mapping_treeTagger_lefff.csv'

#with open(file_path) as f:
#    csv = read_csv(f)

#treeTag = [term for term in csv['TreeTagger'].tolist()] 
#lefff = [term for term in csv['Lefff'].tolist()]

#mapping = {term : lefff[treeTag.index(term)] for term in treeTag}

In [89]:
# tagged = [[t.split('\t')[0], mapping[t.split('\t')[1]]] for t in tagger.tag_text(corpus)]

In [90]:
# Si on veut lemmatiser
### **Lemmatisation** (FrenchLefffLemmatizer)
#https://github.com/ClaudeCoulombe/FrenchLefffLemmatizer

#from french_lefff_lemmatizer.french_lefff_lemmatizer import FrenchLefffLemmatizer
#lemmatizer = FrenchLefffLemmatizer()

#lemmas = []
#for term in tagged:
#    term_l = []
#    if lemmatizer.lemmatize(term[0], term[1]) == []:
#        term_l = (lemmatizer.lemmatize(term[0]), term[1])
    
    # elif type(lemmatizer.lemmatize(term[0], term[1])) == str:
    #     term_l  = (lemmatizer.lemmatize(term[0], term[1]), term[1])

    # else:
    #     term_l = tuple(lemmatizer.lemmatize(term[0], term[1])[0])
    
    # lemmas.append(term_l)

### **Collocations / Phrases / N-Grammes (MWE)**
https://www.kaggle.com/code/alvations/n-gram-language-model-with-nltk/notebook  

In [91]:
def extr_ngrams(tagged):
    ngrammes= list(everygrams(tagged, min_len=1, max_len=12))
    print("Avant filtrage, on a {} ngrammes.".format(len(ngrammes)))
    return ngrammes

In [92]:
ngrammes = extr_ngrams(tagged)

Avant filtrage, on a 44025810 ngrammes.


In [93]:
ngrammes[:15]

[(('MWE_STOP', 'NAM'),),
 (('MWE_STOP', 'NAM'), ('MWE_STOP', 'NAM')),
 (('MWE_STOP', 'NAM'), ('MWE_STOP', 'NAM'), ('MWE_STOP', 'NAM')),
 (('MWE_STOP', 'NAM'),
  ('MWE_STOP', 'NAM'),
  ('MWE_STOP', 'NAM'),
  ('MWE_STOP', 'NAM')),
 (('MWE_STOP', 'NAM'),
  ('MWE_STOP', 'NAM'),
  ('MWE_STOP', 'NAM'),
  ('MWE_STOP', 'NAM'),
  ('recherche', 'VER:pres')),
 (('MWE_STOP', 'NAM'),
  ('MWE_STOP', 'NAM'),
  ('MWE_STOP', 'NAM'),
  ('MWE_STOP', 'NAM'),
  ('recherche', 'VER:pres'),
  ('MWE_STOP', 'NAM')),
 (('MWE_STOP', 'NAM'),
  ('MWE_STOP', 'NAM'),
  ('MWE_STOP', 'NAM'),
  ('MWE_STOP', 'NAM'),
  ('recherche', 'VER:pres'),
  ('MWE_STOP', 'NAM'),
  ('MWE_STOP', 'NAM')),
 (('MWE_STOP', 'NAM'),
  ('MWE_STOP', 'NAM'),
  ('MWE_STOP', 'NAM'),
  ('MWE_STOP', 'NAM'),
  ('recherche', 'VER:pres'),
  ('MWE_STOP', 'NAM'),
  ('MWE_STOP', 'NAM'),
  ('répertoire', 'NOM')),
 (('MWE_STOP', 'NAM'),
  ('MWE_STOP', 'NAM'),
  ('MWE_STOP', 'NAM'),
  ('MWE_STOP', 'NAM'),
  ('recherche', 'VER:pres'),
  ('MWE_STOP', 'NAM'),

### **Extraction des patrons syntaxiques**

*LONG*

In [94]:
def extract_patterns(ngrammes):
    patterns = []
    for ng in ngrammes:
        phrase = [t[0] for t in ng]
        pattern = [t[1] for t in ng]
        patterns.append([phrase, pattern])
    return patterns

In [95]:
phrases = extract_patterns(ngrammes)
# phrases_lemmatized = extract_patterns(ngrammes_lemmatized)

In [ ]:
def freq(phrases):
    return FreqDist([" ".join(t[0]).replace("' ", "'") for t in phrases])

In [ ]:
phrases[:15]

[[['auteurs'], ['NOM']],
 [['auteurs', 'michel'], ['NOM', 'ADJ']],
 [['auteurs', 'michel', 'lavoie'], ['NOM', 'ADJ', 'VER:subp']],
 [['auteurs', 'michel', 'lavoie', 'institut'],
  ['NOM', 'ADJ', 'VER:subp', 'NOM']],
 [['auteurs', 'michel', 'lavoie', 'institut', 'national'],
  ['NOM', 'ADJ', 'VER:subp', 'NOM', 'ADJ']],
 [['auteurs', 'michel', 'lavoie', 'institut', 'national', 'de'],
  ['NOM', 'ADJ', 'VER:subp', 'NOM', 'ADJ', 'PRP']],
 [['auteurs', 'michel', 'lavoie', 'institut', 'national', 'de', 'santé'],
  ['NOM', 'ADJ', 'VER:subp', 'NOM', 'ADJ', 'PRP', 'NOM']],
 [['auteurs',
   'michel',
   'lavoie',
   'institut',
   'national',
   'de',
   'santé',
   'publique'],
  ['NOM', 'ADJ', 'VER:subp', 'NOM', 'ADJ', 'PRP', 'NOM', 'ADJ']],
 [['auteurs',
   'michel',
   'lavoie',
   'institut',
   'national',
   'de',
   'santé',
   'publique',
   'du'],
  ['NOM', 'ADJ', 'VER:subp', 'NOM', 'ADJ', 'PRP', 'NOM', 'ADJ', 'PRP:det']],
 [['auteurs',
   'michel',
   'lavoie',
   'institut',
   'natio

In [ ]:
frequencies = freq(phrases)

In [ ]:
frequencies

FreqDist({'de': 232359, 'la': 118905, 'et': 102194, 'des': 98362, 'les': 95531, "l'": 81879, "d'": 69277, 'à': 67911, 'le': 63621, 'en': 49727, ...})

### **Filtrage** 
On retire les n-grammes qui débutent ou se terminent par un stopword (antidictionnaire)

In [ ]:
# Importer l'antidictionnaire pour filtrer les données

# # Stopwords lemmatisés
# file_path = '../04-filtrage/stopwords_lemmatized.txt'
# with open(file_path, 'r', encoding="utf-8") as f:
#     stopwords_lemmatized = [w.strip('\n').lower() for w in f.readlines()]

# Stopwords fréquents en français (non lemmatisés)
file_path = "../04-filtrage/stopwords.txt"
with open(file_path, 'r', encoding="utf-8") as f:
    stopwords = [t.lower().strip('\n') for t in f.readlines()]


# Stopwords fréquents en anglais (non lemmatisés)
file_path = '../04-filtrage/stop_words_english.txt'
with open(file_path, 'r', encoding="utf-8") as f:
    stopwords += [t.lower().strip('\n') for t in f.readlines()]

*LONG*

In [ ]:
def filtrer_stopwords(x): # On s'assure aussi que le premier terme du ngramme est un NOM pour avoir des syntagmes nominaux
    if lng == 'en':
        nom = 'NN'
    if lng == 'fr':
        nom = 'NOM'
    return [term for term in x if not 'MWE_STOP' in term[0] and not term[0][0] in stopwords and not term[0][-1] in stopwords and not 'NUM' in term[1] and term[1][0] == nom]

In [ ]:
phrases = filtrer_stopwords(phrases)

In [ ]:
phrases

[[['auteurs'], ['NOM']],
 [['auteurs', 'michel', 'lavoie', 'institut'],
  ['NOM', 'ADJ', 'VER:subp', 'NOM']],
 [['auteurs', 'michel', 'lavoie', 'institut', 'national'],
  ['NOM', 'ADJ', 'VER:subp', 'NOM', 'ADJ']],
 [['auteurs', 'michel', 'lavoie', 'institut', 'national', 'de', 'santé'],
  ['NOM', 'ADJ', 'VER:subp', 'NOM', 'ADJ', 'PRP', 'NOM']],
 [['auteurs',
   'michel',
   'lavoie',
   'institut',
   'national',
   'de',
   'santé',
   'publique'],
  ['NOM', 'ADJ', 'VER:subp', 'NOM', 'ADJ', 'PRP', 'NOM', 'ADJ']],
 [['auteurs',
   'michel',
   'lavoie',
   'institut',
   'national',
   'de',
   'santé',
   'publique',
   'du',
   'québec'],
  ['NOM',
   'ADJ',
   'VER:subp',
   'NOM',
   'ADJ',
   'PRP',
   'NOM',
   'ADJ',
   'PRP:det',
   'ADJ']],
 [['institut'], ['NOM']],
 [['institut', 'national'], ['NOM', 'ADJ']],
 [['institut', 'national', 'de', 'santé'], ['NOM', 'ADJ', 'PRP', 'NOM']],
 [['institut', 'national', 'de', 'santé', 'publique'],
  ['NOM', 'ADJ', 'PRP', 'NOM', 'ADJ']],


On retire les n-grammes qui débutent ou se terminent par token dont la longueur est inférieure à 2 caractères ou supérieure à 18 caractères

In [ ]:
def filter_len(x):
    return [term for term in x if \
        len(term[0][0]) > 2 and len(term[0][0]) < 18 and \
        len(term[0][-1]) > 2 and len(term[0][-1]) < 18]

In [ ]:
phrases = filter_len(phrases)

On retire les n-grammes qui apparaissent moins de 5 fois dans le corpus

In [ ]:
def filter_freq(x):
    return [term for term in x if frequencies[" ".join(term[0]).replace("' ", "'")] > 5]

In [ ]:
phrases = filter_freq(phrases)

In [ ]:
phrases = [[" ".join(term[0]).replace("' ", "'"), " ".join(term[1])] for term in phrases]
# phrases_lemmatized = filtrer_phrases(phrases_lemmatized, freq_lemmatized)

In [ ]:
for phrase in phrases:
    phrase.append(frequencies[phrase[0]])

In [ ]:
print("Après filtrage, on a {} occurrences de ngrammes.".format(len(phrases))) 

Après filtrage, on a 2744893 occurrences de ngrammes.


In [ ]:
phrases[:15]

[['auteurs', 'NOM', 1400],
 ['institut', 'NOM', 4235],
 ['institut national', 'NOM ADJ', 2718],
 ['institut national de santé', 'NOM ADJ PRP NOM', 2605],
 ['institut national de santé publique', 'NOM ADJ PRP NOM ADJ', 2598],
 ['institut national de santé publique du québec',
  'NOM ADJ PRP NOM ADJ PRP:det ADJ',
  2529],
 ['santé', 'NOM', 32069],
 ['santé publique', 'NOM ADJ', 10001],
 ['santé publique du québec', 'NOM ADJ PRP:det ADJ', 3258],
 ['institut', 'NOM', 4235],
 ['institut national', 'NOM ADJ', 2718],
 ['institut national de santé', 'NOM ADJ PRP NOM', 2605],
 ['institut national de santé publique', 'NOM ADJ PRP NOM ADJ', 2598],
 ['institut national de santé publique du québec',
  'NOM ADJ PRP NOM ADJ PRP:det NOM',
  2529],
 ['santé', 'NOM', 32069]]

In [ ]:
DataFrame(phrases, columns=["Expression", "Patron syntaxique", "Fréquence"])

,Expression,Patron syntaxique,Fréquence
0,auteurs,NOM,1400
1,institut,NOM,4235
2,institut national,NOM ADJ,2718
3,institut national de santé,NOM ADJ PRP NOM,2605
4,institut national de santé publique,NOM ADJ PRP NOM ADJ,2598
...,...,...,...
2744888,exposition au plomb reliée,NOM PRP:det NOM VER:pper,17
2744889,plomb,NOM,265
2744890,plomb reliée,NOM VER:pper,17
2744891,nouveautés,NOM,883


In [ ]:
def tabCSV(phrases):
    base_path = '../04-filtrage/output/'
    tab = DataFrame(phrases, columns=["Expression", "Patron syntaxique", "Fréquence"]).drop_duplicates()
    tab.sort_values(["Fréquence"], 
                        axis=0,
                        ascending=[False], 
                        inplace=True)


    # file_path = path.join(base_path, tag, tag)    
    # Path(file_path).mkdir(parents=True, exist_ok=True)

    #tab.to_csv(file_path + '_ngrams.csv') - On va seulement garder les collocations significatives comme output

    return tab.values.tolist()

In [ ]:
phrases = tabCSV(phrases)
# phrases_lemmatized = tabCSV(phrases_lemmatized, '_n-grams-lemmatized.csv')

In [ ]:
print("Après filtrage, on a {} ngrammes uniques.".format(len(phrases)))

Après filtrage, on a 123554 ngrammes uniques.


### **Filtrage (Patrons syntaxiques)**  
Lossio-Ventura, J. A., Jonquet, C., Roche, M., & Teisseire, M. (2014). Biomedical Terminology Extraction : A new combination of Statistical and Web Mining Approaches. 421. https://hal-lirmm.ccsd.cnrs.fr/lirmm-01056598

On veut aller extraire les structures syntaxiques les plus courantes dans les MeSH pour filtrer notre corpus selon celles-ci (inspiré de la méthodologie de l'article ci-dessus ; voir le Notebook *Mesh_extract.ipynb*). Pour ce faire, nous allons donc ne sélectionner que les ngrammes qui y correspondent. 

In [ ]:
file_patterns = '../04-filtrage/MeSH/mesh_patterns-fr.csv'

with open (file_patterns, 'r') as f:
    patterns = read_csv(f)
    patterns = patterns['Structure'].tolist() #[:200] # On prend les 200 structures syntaxiques les plus fréquentes dans les MeSH

In [ ]:
def filter_patterns(phrases):
    return [t for t in phrases if t[1] in patterns and not 'NOM NOM' in t[1] and not 'NUM' in t[1]] # 
# terms_lemmatized = [t for t in phrases_lemmatized if t[1] in patterns]

In [ ]:
terms = filter_patterns(phrases)
# terms[:50]

#terms = phrases

In [ ]:
print("Le filtrage syntaxique élimine environ {} % des termes".format(round((len(phrases) - len(terms)) / len(phrases) * 100)))
print("On avait {} ngrammes, ".format(len(phrases)) + "on en a maintenant {}.".format(len(terms)))

Le filtrage syntaxique élimine environ 54 % des termes
On avait 123554 ngrammes, on en a maintenant 57183.


In [ ]:
# import pandas as pd

# def extract_terms(liste_terms):
#     file_path = '../04-filtrage/output/'
#     tab = pd.DataFrame(terms, columns= ["Expression", "Structure syntaxique", "Fréquence"]).drop_duplicates(subset='Expression', keep="last")
#     tab.sort_values(["Fréquence"], 
#                         axis=0,
#                         ascending=[False], 
#                         inplace=True)

#     return 

#     # file_path = path.join(file_path, tag, tag)                    
#     # tab.to_csv(file_path + '_terms.csv')

# extract_terms(terms)
# #extract_terms(terms_lemmatized)

In [ ]:
# terms = phrases

terms_patterns = DataFrame(terms, columns = ["Expression", "Structure syntaxique", "Fréquence"])
terms_patterns = terms_patterns.to_dict('records')
dict_patterns = {}
for term in terms_patterns:
     exp = term['Expression']
     pattern = term['Structure syntaxique']
     dict_patterns[exp] = pattern

### **Filtrage (Collocations statistiquement significatives)** Log-Likelihood Ratio

[Notebook - Collocation extraction methodologies compared](https://notebooks.githubusercontent.com/view/ipynb?azure_maps_enabled=false&browser=chrome&color_mode=auto&commit=33868e847376764d7733cd958986c88dedfaec97&device=unknown&enc_url=68747470733a2f2f7261772e67697468756275736572636f6e74656e742e636f6d2f746f64642d636f6f6b2f4d4c2d596f752d43616e2d5573652f333338363865383437333736373634643737333363643935383938366338386465646661656339372f70726f626162696c69737469635f6c616e67756167655f6d6f64656c696e672f636f6c6c6f636174696f6e5f65787472616374696f6e732e6970796e62&enterprise_enabled=false&logged_in=false&nwo=todd-cook%2FML-You-Can-Use&path=probabilistic_language_modeling%2Fcollocation_extractions.ipynb&platform=android&repository_id=167140788&repository_type=Repository&version=102)

On applique un test d'hypothèse statistique aux n-grammes sur lesquels une probabilité a été mesurée (Log-likelihood ratio) - seuls les n-grammes dont le test est significatif seront conservés.
On considère que l'apparition de ces collocations dans notre corpus n'est pas dûe au hasard.

In [ ]:
def loglikelihood_ratio(c_prior, c_n, c_ngram, N):
    """
    Compute the ratio of two hypotheses of likelihood and return the ratio.
    The formula here and test verification values are taken from 
    Manning & Schūtze _Foundations of Statistical Natural Language Processing_ p.172-175
    Parameters:
    c_prior: count of word 1 if bigrams or count of [w1w2 .. w(n-1)] if ngram
    c_n : count of word 2 if bigrams or count of wn if ngram
    c12: count of bigram (w1, w2) if bigram or count of ngram if ngram
    N: the number of words in the corpus
    """

    p = c_n / N
    p1 = c_ngram / c_prior
    p2 = (c_n - c_ngram) / (N - c_prior)   
    # We proactively trap a runtimeWarning: divide by zero encountered in log,
    # which may occur with extreme collocations
    import warnings
    with warnings.catch_warnings(): # this will reset our filterwarnings setting
        warnings.filterwarnings('error')
        try:
            return (np.log(binom.pmf(c_ngram, c_prior, p)) 
                    + np.log(binom.pmf(c_n - c_ngram, N - c_prior, p)) 
                    - np.log(binom.pmf(c_ngram, c_prior, p1) )
                    - np.log(binom.pmf(c_n - c_ngram, N - c_prior, p2)))             
        except Warning:
            return np.inf 

In [ ]:
len_prior = len(terms)
print("Au départ, on a {} ngrammes.".format(len_prior))

Au départ, on a 57183 ngrammes.


In [ ]:
# Pour le calcul des probabilités, on a besoin de traiter séparément les ngrammes selon la valeur de n
N = len(tokens)
fd_tokens = nltk.FreqDist(tokens)

In [ ]:
fd_tokens

FreqDist({'de': 232359, 'la': 118905, 'et': 102194, 'des': 98362, 'les': 95531, "l'": 81879, "d'": 69277, 'à': 67911, 'le': 63621, 'en': 49727, ...})

In [ ]:
def llr_ngrammes(n):
    llr = []

    for i in range(2, n+1):
        ngrammes = set([tuple(tokenizer_re.tokenize(term[0])) for term in terms if len(tokenizer_re.tokenize(term[0])) == i])
        fd = nltk.FreqDist(ngrams(tokens, n=i))
        fd_prior = nltk.FreqDist(ngrams(tokens, n=i-1))
        
        for t in ngrammes:
            c_prior = fd_prior[t[:i-1]] # Antécédent = P(w1w2..w_n-1) (si on considère que P(w1w2...wn) = P(wn) | P(w1w2...w_n-1)
            c_n = fd_tokens[t[i-1]]     # Dernier mot du ngramme  P(wn)
            c_ngram = fd[t]             # Le ngramme lui-même P(w1w2w3..wn)

            try:
                res = -2 * loglikelihood_ratio(c_prior, c_n, c_ngram, N)
                p_value = chi2.sf(res, 1) # 1 degrees of freedom
                #if res == float('-inf') :
                #    res = 50000

                if p_value < 0.1 or (res == float('-inf')):
                    llr.append({'Collocation' : " ".join(t).replace("' ", "'"), 'Structure syntaxique': dict_patterns[" ".join(t).replace("' ", "'")], 'Fréquence' : c_ngram, 'LLR': res, 'p-value': p_value})
            
            except Exception as e:
                print(t, str(e))
            

    return llr


In [ ]:
terms = llr_ngrammes(12)

("o'", 'bomsawin') division by zero
("p'", 's') division by zero
("p'", 's', 'model') division by zero


In [ ]:
terms
#terms = [{'Collocation' : t[0], 'Structure syntaxique': t[1], 'Fréquence' : t[2]} for t in terms]

[{'Collocation': 'santé cognitive',
  'Structure syntaxique': 'NOM ADJ',
  'Fréquence': 152,
  'LLR': 1219.3854357833106,
  'p-value': 3.735348196841563e-267},
 {'Collocation': 'québec publié',
  'Structure syntaxique': 'NOM VER:pper',
  'Fréquence': 8,
  'LLR': 8.537928213433494,
  'p-value': 0.0034782130903503765},
 {'Collocation': 'polyarthrite rhumatoïde',
  'Structure syntaxique': 'NOM ADJ',
  'Fréquence': 6,
  'LLR': 159.45513785631442,
  'p-value': 1.4883167967224642e-36},
 {'Collocation': 'seringues stériles',
  'Structure syntaxique': 'NOM ADJ',
  'Fréquence': 12,
  'LLR': 194.77051841930484,
  'p-value': 2.8913182914016565e-44},
 {'Collocation': 'méthodologie utilisée',
  'Structure syntaxique': 'NOM VER:pper',
  'Fréquence': 26,
  'LLR': 312.9089102349946,
  'p-value': 5.076303287846798e-70},
 {'Collocation': 'défavorisation prédominants',
  'Structure syntaxique': 'NOM ADJ',
  'Fréquence': 7,
  'LLR': 109.42488739444704,
  'p-value': 1.3097474775517456e-25},
 {'Collocation'

In [ ]:
print('Après avoir calculé le log-likelihood ratio, on a retiré {} collocations qui n\'étaient pas statistiquement significatives.'.format(len_prior - len(terms)))
print('Ça représente environ {} % de nos n-grammes.'.format(round((len_prior - len(terms)) / len_prior *100 )))

Après avoir calculé le log-likelihood ratio, on a retiré 13841 collocations qui n'étaient pas statistiquement significatives.
Ça représente environ 24 % de nos n-grammes.


In [ ]:
DataFrame(terms).sort_values(by = "Fréquence", ascending=False)

,Collocation,Structure syntaxique,Fréquence,LLR,p-value
6136,santé publique,NOM ADJ,10001,-inf,1.000000e+00
22414,santé publique du québec,NOM ADJ PRP:det NOM,3258,-inf,1.000000e+00
7755,institut national,NOM ADJ,2718,-inf,1.000000e+00
23786,institut national de santé,NOM ADJ PRP NOM,2605,-inf,1.000000e+00
31470,institut national de santé publique,NOM ADJ PRP NOM ADJ,2598,-inf,1.000000e+00
...,...,...,...,...,...
26051,tabac chez les fumeurs,NOM PRP DET:ART NOM,6,89.478240,3.100299e-21
37022,collisions avec blessures graves ou mortelles,NOM PRP NOM ADJ KON ADJ,6,126.867532,1.985967e-29
15915,creating a healthy,NOM VER:pres NOM,6,85.661676,2.135111e-20
32439,capsules d'informations du service,NOM PRP NOM PRP:det NOM,6,101.599299,6.796936e-24


### **Filtrage - Fréquence documentaire**
** Il y aurait quelque chose à modifier ici ; en tokenisant, on supprimer les frontières entre les mots qui sont des tirets ou autres caractères qu'un espace ou un apostrophe ; ça fait en sorte qu'on a une fréquence documentaire de 0 pour les ngrammes qui n'ont plus la forme exacte qu'ils avaient dans le corpus orginal. **

*LONG*  
Est-ce qu'il y a moyen de rendre ça plus efficace ?

In [ ]:
dfs = {term['Collocation']: len([doc for doc in text if term['Collocation'] in doc]) for term in terms}

In [ ]:
max_df = round(0.95 * nb_docs)        # Pour rejeter les termes qui se retrouvent dans plus de 95% des documents 
max_df

4708

In [ ]:
test = dfs[max(dfs, key=dfs.get)] # Voir quelle est la fréquence documentaire maximale qu'on retrouve

In [ ]:
zeros = {term for term in dfs if dfs[term] == 0}
dfs_100 = {term for term in dfs if dfs[term] >= max_df}

In [ ]:
dfs_100

set()

In [ ]:
terms = [t for t in terms if dfs[t['Collocation']] < max_df ]

In [ ]:
#colloc = [term['Collocation'] for term in terms]
#len(colloc)

In [ ]:
#for term in terms:
 #   term['DF'] = dfs[term['Collocation']] 
DataFrame(terms).sort_values(by="Fréquence", ascending=False)

,Collocation,Structure syntaxique,Fréquence,LLR,p-value
6136,santé publique,NOM ADJ,10001,-inf,1.000000e+00
22414,santé publique du québec,NOM ADJ PRP:det NOM,3258,-inf,1.000000e+00
7755,institut national,NOM ADJ,2718,-inf,1.000000e+00
23786,institut national de santé,NOM ADJ PRP NOM,2605,-inf,1.000000e+00
31470,institut national de santé publique,NOM ADJ PRP NOM ADJ,2598,-inf,1.000000e+00
...,...,...,...,...,...
26051,tabac chez les fumeurs,NOM PRP DET:ART NOM,6,89.478240,3.100299e-21
37022,collisions avec blessures graves ou mortelles,NOM PRP NOM ADJ KON ADJ,6,126.867532,1.985967e-29
15915,creating a healthy,NOM VER:pres NOM,6,85.661676,2.135111e-20
32439,capsules d'informations du service,NOM PRP NOM PRP:det NOM,6,101.599299,6.796936e-24


In [ ]:
df = DataFrame(terms)
df.sort_values(['Fréquence'], 
            axis=0,
            ascending=[False], 
            inplace=True)
if lng == 'en':
    acteur = acteur + '_' + lng
if tag:
    output_path = path.join('../04-filtrage/output/', acteur + '_' + tag + '_significant-collocations.csv') 
else:
    output_path = path.join('../04-filtrage/output/', acteur + '_significant-collocations.csv') 
df.to_csv(output_path)

In [ ]:
list_terms = [term['Collocation'] for term in terms]

In [ ]:
terms = DataFrame(terms)

### **KWIC (Keyword in Context)**
Termes d'intérêt : 
- « Programme »
- « Plan »
- « Service(s) de » 
- « Intervenant(e) en »
- « Professionnel de »
- « Institut (du/de) »
- « Groupe de recherche en »
- « Personne »
- « Infirmière (en) »

In [ ]:
# Dans notre cas on veut que ça débute par le mot-clé donc le contexte est un peu plus simple
# penser à généraliser avec des expressions régulières
kw = ['programme', 'plan ', 'service', 'intervenant', 'infirmière en', 'institut', 'groupe de recherche', 'personne', 'maladie']

ngrammes_kwic = [" ".join([t[0] for t in ng]).replace("' ", "'") for ng in ngrammes]

In [ ]:
extrant = DataFrame(columns=['Mot-clé','Concordance', 'Fréquence'])
kwic = {w : [] for w in kw} 

In [ ]:
for t in ngrammes_kwic: # on pourrait aussi chercher dans les terms, mais on perd certains termes d'intérêt avec le filtrage syntaxique
    for w in kw:
        if t.startswith(w):
            kwic[w].append(t)

In [ ]:
kwic = {term: FreqDist(kwic[term]) for term in kwic}

In [ ]:
for term in kw:
    df = DataFrame(kwic[term].items(), columns=['Concordance', "Fréquence"])
    df.sort_values(["Fréquence"], 
        axis=0,
        ascending=[False], 
        inplace=True)

    df.insert(0, 'Mot-clé', term)
    extrant = concat([extrant, df])


extrant = extrant[extrant['Fréquence'] > 30] 

file_path = '../04-filtrage/output/'
file_path = path.join(file_path, tag, tag)


#extrant.to_csv(file_path + '_KWIC' +'.csv')

extrant

,Mot-clé,Concordance,Fréquence
0,programme,programme,4340
102,programme,programme de,1680
46,programme,programmes,1540
658,programme,programme de surveillance,904
47,programme,programmes de,636
...,...,...,...
891,maladie,maladies entériques,33
1145,maladie,maladies d'origine hydrique signalées,32
1153,maladie,maladies d'origine hydrique et,31
2849,maladie,maladies transmises par la tique ixodes scapul...,31


### **Extraction de termes MeSH**

In [ ]:
from nltk.tokenize import MWETokenizer
file_path = '../04-filtrage/MeSH/mesh-fr.txt'

with open (file_path, 'r', encoding='utf-8') as f:
    mesh = [tuple(tokenizer_re.tokenize(w)) for w in f.readlines()]
    tokenizer_mesh = MWETokenizer(mesh, separator= ' ')
    mesh = [tokenizer_mesh.tokenize(w)[0].lower() for w in mesh]
    mesh = [w for w in mesh if len(w.split()) > 1] # On ne retient que les termes complexes
    #mesh = [tuple(t.strip('.').lower().split()) for t in f.readlines()]

In [ ]:
# extr_mesh = tokenizer_mesh.tokenize([t['Collocation'] for t in terms])

# **MODIF**

In [ ]:
# MODIFICATION À APPORTER : dans l'extrant où on voit les termes mesh, ajouter la fréquence

In [ ]:
termes_mesh = []

for t in extr_mesh:
    if t in mesh:
        termes_mesh.append(t)

NameError: name 'extr_mesh' is not defined

In [ ]:
file_path = '../04-filtrage/output/'
if sous_corpus:
    file_path = path.join(file_path, acteur, tag, tag)

else :
    file_path = path.join(file_path, acteur, acteur)

df = DataFrame(termes_mesh)
df.to_csv(file_path + '_MeSH.csv')

: 

### **Extraction de termes SNOMED**

In [ ]:
from nltk.tokenize import MWETokenizer
file_path = '../04-filtrage/SNOMED_fr.csv'

with open(file_path, 'r', encoding='utf-8') as f:
    sm = read_csv(f, sep=';')
    sm = list(dict.fromkeys([str(t).strip().lower() for t in sm['term'].tolist()]))

    sm = [tuple(tokenizer_re.tokenize(w)) for w in sm if len(w.split()) > 1]
    tokenizer_sm = MWETokenizer(sm, separator = ' ')

    sm = [tokenizer_sm.tokenize(w)[0].lower() for w in sm]

: 

In [ ]:
extr_sm = tokenizer_sm.tokenize([t[0] for t in terms])

: 

In [ ]:
termes_sm = []

for t in extr_sm:
    if t in sm:
        termes_sm.append(t)

: 

In [ ]:
file_path = '../04-filtrage/output' 
if sous_corpus:
    file_path = path.join(file_path, acteur, tag, tag) 

else :
    file_path = path.join(file_path, acteur, acteur)


df = DataFrame(termes_sm)

df.to_csv(file_path + '_SNOMED.csv')

: 

: 